In [13]:
import pandas as pd
import numpy as np
import datetime 

data = pd.read_csv(r'C:\Users\nadia\Dropbox\Internship - Kairaktidi\data-ypen\ΠΕΙΡΑΙΑΣ-I-PIR\PIRPYTHON.csv')

print(data)

            DATETIME   C6H6   NO2  SO2   PM10  PM2.5    O3
0      01-01-20 0:00   9.98  57.0  8.0  133.0  112.0   8.0
1      01-01-20 1:00  11.15  56.0  8.0  142.0  120.0  10.0
2      01-01-20 2:00  11.71  53.0  9.0  197.0  175.0   6.0
3      01-01-20 3:00   9.26  46.0  7.0  146.0  123.0   6.0
4      01-01-20 4:00   9.68  45.0  8.0  133.0  117.0   5.0
...              ...    ...   ...  ...    ...    ...   ...
8779  31-12-20 19:00   0.77  32.0  4.0   44.0   12.0  40.0
8780  31-12-20 20:00   1.15  52.0  5.0   46.0   16.0  26.0
8781  31-12-20 21:00   0.61  36.0  5.0   47.0   21.0  43.0
8782  31-12-20 22:00   0.40  52.0  4.0   25.0    6.0  28.0
8783  31-12-20 23:00   0.48  41.0  6.0   27.0    8.0  38.0

[8784 rows x 7 columns]


In [14]:
data['DATETIME']=pd.to_datetime(data['DATETIME'])
data.groupby(by=data['DATETIME'].dt.date).count()

,DATETIME,C6H6,NO2,SO2,PM10,PM2.5,O3
DATETIME,,,,,,,
2020-01-01,24,24,24,24,24,24,24
2020-01-02,24,24,24,24,24,24,24
2020-01-03,24,0,24,24,24,11,24
2020-01-04,24,24,24,24,24,24,24
2020-01-05,24,0,24,24,24,24,24
...,...,...,...,...,...,...,...
2020-12-27,24,24,24,24,24,24,24
2020-12-28,24,24,24,24,24,24,24
2020-12-29,24,24,24,24,23,24,24


In [15]:
data['date'] = data['DATETIME'].dt.date
print(data)

                DATETIME   C6H6   NO2  SO2   PM10  PM2.5    O3        date
0    2020-01-01 00:00:00   9.98  57.0  8.0  133.0  112.0   8.0  2020-01-01
1    2020-01-01 01:00:00  11.15  56.0  8.0  142.0  120.0  10.0  2020-01-01
2    2020-01-01 02:00:00  11.71  53.0  9.0  197.0  175.0   6.0  2020-01-01
3    2020-01-01 03:00:00   9.26  46.0  7.0  146.0  123.0   6.0  2020-01-01
4    2020-01-01 04:00:00   9.68  45.0  8.0  133.0  117.0   5.0  2020-01-01
...                  ...    ...   ...  ...    ...    ...   ...         ...
8779 2020-12-31 19:00:00   0.77  32.0  4.0   44.0   12.0  40.0  2020-12-31
8780 2020-12-31 20:00:00   1.15  52.0  5.0   46.0   16.0  26.0  2020-12-31
8781 2020-12-31 21:00:00   0.61  36.0  5.0   47.0   21.0  43.0  2020-12-31
8782 2020-12-31 22:00:00   0.40  52.0  4.0   25.0    6.0  28.0  2020-12-31
8783 2020-12-31 23:00:00   0.48  41.0  6.0   27.0    8.0  38.0  2020-12-31

[8784 rows x 8 columns]


# C6H6

In [16]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας

data.replace('', np.nan).count()
c = data['C6H6'].count()
print("Οι συνολικές τιμές είναι", data.shape[0] )
print("Οι έγκυρες τιμές είναι", c)
if c > (0.9*data.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Οι συνολικές τιμές είναι 8784
Οι έγκυρες τιμές είναι 7029
Δεν ικανοποιείται ο έλεγχος πληρότητας


In [17]:
# Έλεγχος οριακής τιμής στο ημερολογιακό έτος (5 μg/m3)

print("Μέση παρατηρούμενη ετήσια τιμή:", round(data['C6H6'].mean(), 2), "μg/m3")

if data['C6H6'].mean() > 5:
    print("Γίνεται υπέρβαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου (5 μg/m3)")
    

Μέση παρατηρούμενη ετήσια τιμή: 2.04 μg/m3
Δεν παρατηρείται υπέρβαση του ορίου (5 μg/m3)


# NO2

In [18]:
# Ωριαία οριακή τιμή (200μg/m3 έως 18 φορές τον χρόνο)

print("Για τις ωριαίες τιμές ικανοποιείται ο έλεγχος πληρότητας των 45 λεπτών")
if data.loc[data.NO2 < 200, 'NO2'].count() > 18 :
     print("Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (200μg/m3) για πάνω από 18 φορές το έτος")
else:
    print("Γίνεται υπέρβαση του ορίου")

Για τις ωριαίες τιμές ικανοποιείται ο έλεγχος πληρότητας των 45 λεπτών
Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (200μg/m3) για πάνω από 18 φορές το έτος


In [19]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας για την ετήσια οριακή τιμή 

#print(sum(pd.isnull(data['NO2'])))
data.replace('', np.nan).count()
s = data['NO2'].count()
print("Οι συνολικές τιμές είναι", data.shape[0] )
print("Οι έγκυρες τιμές είναι", s)
if s > (0.9*data.shape[0]):
    print("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    print("Δεν ικανοποιείται ο έλεγχος πληρότητας")

Οι συνολικές τιμές είναι 8784
Οι έγκυρες τιμές είναι 8765
Ικανοποιείται ο έλεγχος πληρότητας


In [20]:
# Οριακή τιμή στο ημερολογ. έτος = 40 μg/m3
if data['NO2'].mean() > 40:
    print("Γίνεται υπέρβαση του ορίου,", round(data['NO2'].mean(), 1), "μg/m3 με όριο τα 40 μg/m3")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου ( 40 μg/m3)")

Γίνεται υπέρβαση του ορίου, 55.0 μg/m3 με όριο τα 40 μg/m3


# SO2

In [21]:
# Ωριαία οριακή τιμή (350μg/m3 έως 24 φορές τον χρόνο)

print("Για τις ωριαίες τιμές ικανοποιείται ο έλεγχος πληρότητας των 45 λεπτών","\n")

if data.loc[data.SO2 < 350, 'NO2'].count() > 24 :
     print("Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (350μg/m3) για πάνω από 24 φορές το έτος")
else:
    print("Γίνεται υπέρβαση του ορίου")

print("Οι συνολικές υπερβάσεις ήταν:" ,data.loc[data.SO2 > 350, 'NO2'].count())

Για τις ωριαίες τιμές ικανοποιείται ο έλεγχος πληρότητας των 45 λεπτών 

Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (350μg/m3) για πάνω από 24 φορές το έτος
Οι συνολικές υπερβάσεις ήταν: 0


In [37]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας για την ημερήσια οριακή τιμή

grouped_single = data.groupby('date').agg({'SO2': ['count','max'], 'PM10': ['count','max','mean']})
grouped_single.replace('', np.nan).count()

b = grouped_single['SO2']['count'].le(18) #άκυρες και μέρες με λιγότερο από 18 ώρες μαζί

print("Οι συνολικές τιμές είναι", grouped_single.shape[0] )
print("Οι έγκυρες τιμές είναι", len(grouped_single['SO2']) -b.sum())
    
grouped_single.head(75)

Οι συνολικές τιμές είναι 366
Οι έγκυρες τιμές είναι 354


SO2        PM10                  
           count   max count    max       mean
date                                          
2020-01-01    24  14.0    24  197.0  63.500000
2020-01-02    24  24.0    24  102.0  32.916667
2020-01-03    24  20.0    24   54.0  28.583333
2020-01-04    24   7.0    24   51.0  43.333333
2020-01-05    24   9.0    24   27.0  17.500000
...          ...   ...   ...    ...        ...
2020-03-11    24  17.0    24   87.0  50.583333
2020-03-12    24  12.0    24   50.0  28.375000
2020-03-13    24  13.0    24   29.0  18.208333
2020-03-14    24  31.0    24   66.0  33.541667
2020-03-15     0   NaN    24   68.0  34.458333

[75 rows x 5 columns]

In [23]:
# Ημερήσια οριακή τιμή (125μg/m3 έως 3 φορές τον χρόνο)

m = grouped_single['SO2']['max'].gt(125)
if m.sum()>3:
    print("Παρατηρείται υπέρβαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")
    
print("Συνολικές υπερβάσεις που παρατηρήθηκαν:", m.sum())

Δεν παρατηρείται υπέρβαση του ορίου
Συνολικές υπερβάσεις που παρατηρήθηκαν: 0


# PM10

In [35]:
p = grouped_single['PM10']['count'].le(18)

print("Οι συνολικές τιμές είναι", grouped_single.shape[0] )
print("Οι έγκυρες τιμές είναι", len(grouped_single['PM10']) - p.sum())

grouped.head(75)

Οι συνολικές τιμές είναι 366
Οι έγκυρες τιμές είναι 321


PM10       
           count    max
date                   
2020-01-01    24  197.0
2020-01-02    24  102.0
2020-01-03    24   54.0
2020-01-04    24   51.0
2020-01-05    24   27.0
...          ...    ...
2020-03-11    24   87.0
2020-03-12    24   50.0
2020-03-13    24   29.0
2020-03-14    24   66.0
2020-03-15    24   68.0

[75 rows x 2 columns]

In [38]:
# Οριακή τιμή ημέρας = 50μg/m3 έως 35 φορές το έτος

pm2 = grouped_single['PM10']['mean'].gt(50) #(εδώ θέλω την μέση τιμή της μέρας ή την μέγιστη;)
if pm2.sum() > 35:
    print("Παρατηρείται υπέρβαση του ορίου")
else:
    print("Δεν παρατηρείται υπέρβαση του ορίου")
    
print("Συνολικές υπερβάσεις που παρατηρήθηκαν:", pm2.sum())

Δεν παρατηρείται υπέρβαση του ορίου
Συνολικές υπερβάσεις που παρατηρήθηκαν: 27
